# Create a TOF lookup table for ESTIA

This notebook recreates the TOF lookup table that is used to compute time-of-flight from time-of-arrival for the ESTIA instrument.

To reproduce the lookup table you will need to provide some instrument parameters like pulse stride and chopper position.

In [ ]:
#%pip install tof

In [ ]:
#%matplotlib ipympl
import scipp as sc
import scipp.constants
from ess.reduce import time_of_flight
from ess.reduce.nexus.types import AnyRun
from scippneutron.chopper import DiskChopper

In [ ]:
# Parameters from the ESTIA McStas file:
# https://git.esss.dk/dmsc-instrumentmodels/estia/-/blob/main/mcstas-master/simulation/Estia_baseline.instr
min_wavelength = sc.scalar(3.75, unit='angstrom')
chopper_position = sc.scalar(10.895, unit='m')
max_velocity = (sc.constants.h / sc.constants.m_n / min_wavelength).to(unit='m/s')
delay = chopper_position / max_velocity
pulse_stride = 1
frequency = -sc.scalar(14., unit='Hz') / pulse_stride

disk_choppers = {
    "fc": DiskChopper(
        frequency=frequency,
        beam_position=sc.scalar(0.0, unit="deg"),
        phase=delay * frequency * sc.scalar(360, unit="deg"),
        axle_position=sc.vector(value=[0, 0., chopper_position.value], unit=chopper_position.unit),
        slit_begin=sc.array(
            dims=["cutout"],
            values=[
                0.
            ],
            unit="deg",
        ),
        slit_end=sc.array(
            dims=["cutout"],
            values=[
                98.
            ],
            unit="deg",
        ),
    ),
}
disk_choppers['fc']

In [ ]:
wf = time_of_flight.TofLookupTableWorkflow()

wf[time_of_flight.LtotalRange] = sc.scalar(35., unit="m"), sc.scalar(45.0, unit="m")
wf[time_of_flight.NumberOfSimulatedNeutrons] = 5000_000
wf[time_of_flight.SourcePosition] = sc.vector([0, 0, 0], unit='m')
wf[time_of_flight.DiskChoppers[AnyRun]] = disk_choppers
wf[time_of_flight.DistanceResolution] = sc.scalar(0.05, unit="m")
wf[time_of_flight.TimeResolution] = sc.scalar(250.0, unit='us')
wf[time_of_flight.LookupTableRelativeErrorThreshold] = 0.06
wf[time_of_flight.PulsePeriod] = 1.0 / sc.scalar(14.0, unit="Hz")
wf[time_of_flight.PulseStride] = pulse_stride
wf[time_of_flight.PulseStrideOffset] = 0

In [ ]:
%%time
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
# This is what the relationship between time-of-arrival and time-of-flight looks like at 40m (at the detector).
table['distance', 100].plot()

In [ ]:
table.plot()

In [ ]:
table.save_hdf5(f'estia-tof-lookup-table-pulse-stride-{pulse_stride}.h5')